In [54]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils, models
from torchsummary import summary
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from PIL import Image

In [55]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [56]:
model = models.resnet18(pretrained=True).to(device)

# Freezing the base model layers to prevent retraining
for param in model.parameters():
    param.requires_grad = False

C:\Users\cardon\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\cardon\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Notes on the Resnet Model:

** convert images to b&w
- Requires input images dimensions (256,256) ** resize our images
- My additional fully connected layer needs dimensions (2048,10) - 10 for the 10 classes for the 10 style types (--)
- Image preprocessing requires:
  1. (224,224) center crop
  2. image is normalized with mean = 255*[0.485, 0.456, 0.406] and
  std = 255*[0.229, 0.224, 0.225]
  3. transpose it from HWC to CHW layout
- Post-processing involves calculating the softmax probability scores for each class

In [57]:
classes = 4
model.fc = torch.nn.Linear(512, classes).to(device)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [58]:
loss_fn = torch.nn.NLLLoss # multi-class classification model loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 

In [59]:
class MyDataset(Dataset):
    def __init__(self,
                 csv_file,      # images could be provided with in a series of directories
                 root_dir,     # images could be provided as a list as well
                 transform = transforms.ToTensor()):  # provide transformation to apply to each image
      """
      Organize the images and the associated labels into two lists.  Potentially create additional
      lists if more complicated information is need.  Important note: images are NOT
      read and stored in this initializer.  They are read in __getitem__ as needed.
      """
      self.csv_file = csv_file # path of csv file
      self.root_dir = root_dir # directory the photos are in
      self.images = pd.read_csv(self.csv_file)
      # Record the transform that may need to be applied.
      self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        '''
        Return a tuple with the data, ground truth label, and any other data
        associated with a single image.
        '''
        img_name = self.images.iloc[idx, 0] # name of image in 1st column
        img_path = os.path.join(self.root_dir, img_name)
        im = Image.open(img_path)

        if self.transform is not None:
            im = self.transform(im)

        """
        label encodes season
        season = {
            0: 'spring'
            1: 'summer'
            2: 'fall'
            3: 'winter'  
        }
        """
        label = self.images.iloc[idx, 1]
        label = torch.tensor(label, dtype=torch.float32)

        return im, label

In [60]:
image_transforms = transforms.Compose([transforms.Grayscale(num_output_channels=1), \
                                      transforms.ToTensor(), transforms.Resize((224, 224)), \
                                       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dataset = MyDataset(csv_file='./filtered_style_stats.csv',
                    root_dir='../crop-data/',
                    transform=image_transforms)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.7, 0.15, 0.15], generator=torch.Generator())

In [70]:
# Referenced from 
# https://towardsdatascience.com/pytorch-basics-sampling-samplers-2a0f29f0bf2a

def get_class_distribution(dataset_obj):
    count_dict = {          \
        'spring': 0,        \
        'summer': 0,        \
        'fall':   0,        \
        'winter': 0         \
    } # type: ignore
    
    for idx in range(len(dataset_obj)):
        element = dataset_obj[idx]
        print(element)
        y_lbl = element[1]
        y_lbl = idx2class[y_lbl]
        count_dict[y_lbl] += 1
    return count_dict

In [72]:
#### CHECKING ITS DECENT ####
total_data = len(train_dataset) + len(val_dataset) + len(test_dataset)
print(f"Length of total data: {total_data}")
print(f"Length of train_dataset: {len(train_dataset)}; {(len(train_dataset)/total_data)*100:.2f}%")
print(f"Length of val_dataset: {len(val_dataset)}; {(len(val_dataset)/total_data)*100:.2f}%")
print(f"Length of test_dataset: {len(test_dataset)}; {(len(test_dataset)/total_data)*100:.2f}%\n")
print(f"Class Distribution of train_dataset: {get_class_distribution(train_dataset)}")
print(f"Class Distribution of val_dataset: {get_class_distribution(val_dataset)}")
print(f"Class Distribution of test_dataset: {get_class_distribution(test_dataset)}")

Length of total data: 279203
Length of train_dataset: 195443; 70.00%
Length of val_dataset: 41880; 15.00%
Length of test_dataset: 41880; 15.00%



FileNotFoundError: [Errno 2] No such file or directory: '../crop-data/bryy-93206.jpg'

In [ ]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(train_dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
'''         0: 'spring'
            1: 'summer'
            2: 'fall'
            3: 'winter' ''' 
def test(dataloader, model, loss_fn, incorrect_examples, correct_examples):
    size = len(test_dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():  # Why do we do this? - we don't want to adjust the gradient & makes it faster
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            fix = torch.tensor((), dtype=torch.float32)
            for val in pred.argmax(1):
              if val == 0:
                item = torch.tensor([1,0], dtype=torch.float32).unsqueeze(0)
                fix = torch.cat((fix, item), dim=0)
              elif val == 1:
                item = torch.tensor([0,1], dtype=torch.float32).unsqueeze(0)
                fix = torch.cat((fix, item), dim=0)
              else:
                item = torch.tensor([1,1], dtype=torch.float32).unsqueeze(0)
                fix = torch.cat((fix, item), dim=0)
            fix = fix.to(device)
            correct += ((fix == y).type(torch.float).sum().item()) / 2        # needed to adjust the sum after doubling the size of the tensor

            if torch.all(torch.eq(fix, y)) and len(correct_examples) < 6:
                correct_examples.append(X.cpu())
            if (not torch.all(torch.eq(fix, y))) and len(incorrect_examples) < 6:
                incorrect_examples.append(X.cpu())

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct_examples, incorrect_examples

In [ ]:
def test(dataloader, model, loss_fn, incorrect_examples, correct_examples):
    size = len(test_dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad(): 
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            val = pred.argmax(1).to(device) 
            correct += ((val == y).type(torch.float).sum().item()) 

            if torch.all(torch.eq(val, y)) and len(correct_examples) < 6:
                correct_examples.append(X.cpu())
            if (not torch.all(torch.eq(val, y))) and len(incorrect_examples) < 6:
                incorrect_examples.append(X.cpu())

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct_examples, incorrect_examples

In [ ]:
epochs = 50

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    incorrect_examples = []
    correct_examples = []
    correct_examples, incorrect_examples = test(test_dataloader, model, loss_fn, incorrect_examples, correct_examples)
print("Done!\n")